# Lab 2: Interactive 3D Visualization and Coordinate Systems, Point Clouds and Voxel Rendering

## Part 1: Interactive 3D Visualization and Coordinate Systems

In this lab, you will learn how to construct interactive 3D scenes using helper tools such as axesHelper, 
explore transformations using the model_matrix_timeseries, and understand camera controls via interactive demos.
We will explore the representation and manipulation of point clouds and voxel-based data structures. 
Students will experiment with interactive tools to edit, visualize, and compress 3D data.

Learning objectives:
- After completing this lab, students will be able to:
- Understand 3D coordinate systems and camera views.
- Use k3d for rendering and manipulating 3D scenes in Jupyter notebooks.
- Create interactive visualizations with camera control and transformation animations.
- Interpret and visualize transformation matrices applied to objects in real time.
- Load, render, and interact with point cloud datasets.
- Use voxel-based visualization techniques.
- Implement callbacks for dynamic interaction with voxel data.

**K3D** is an interactive 3D visualization library for Python, designed to work seamlessly within Jupyter Notebooks. It leverages WebGL through a custom JavaScript frontend to provide real-time rendering of complex 3D scenes directly in the browser. K3D supports a wide range of 3D primitives such as point clouds, meshes, lines, vectors, and volumes, making it ideal for scientific computing, education, and visual analytics. One of its standout features is the ability to interactively rotate, zoom, and pan scenes, as well as dynamically update geometries and attributes through Python code. This makes K3D particularly powerful for creating visual explanations, animations, and hands-on experiments in computer graphics courses.

In [1]:
pip install k3d

In [3]:
import k3d
import numpy as np
import math
from numpy import sin,cos,pi
from ipywidgets import interact, IntSlider

## Coordinate System and Axes Helper

As already seen in Lab 1, in 3D computer graphics, a coordinate system defines how objects are positioned and oriented in space. Most commonly, a right-handed Cartesian coordinate system is used, where each point in space is described by three values (x, y, z). The X-axis typically points to the right, the Y-axis upward, and the Z-axis outward (or inward, depending on the convention). Understanding this system is fundamental for modeling, transforming, and visualizing 3D objects. 

An axes helper is a visual aid that displays the orientation of these three axes, often using colored lines (e.g., red for X, green for Y, and blue for Z). It helps users interpret the spatial arrangement of objects in a scene, especially when rotating the camera or applying transformations. In visualization tools like K3D, manually adding axes using lines serves as a custom implementation of an axes helper, enabling better spatial awareness and easier debugging of 3D scenes.

### Custom Axis Labels in K3D

The line `plot = k3d.plot(axes=['\\alpha', '\\beta', '\\gamma'])` creates a 3D plot using the K3D library with **custom axis labels**. Instead of the default `x`, `y`, and `z` labels, this example uses Greek letters: α (alpha), β (beta), and γ (gamma). These labels are specified using LaTeX-style strings, which K3D supports for mathematical formatting. Custom axis labels are especially useful when visualizing domain-specific data, such as angular parameters, tensor components, or coordinate transformations in physics or engineering, where traditional x/y/z labels may not be meaningful. Calling `plot.display()` renders the interactive 3D plot in the Jupyter notebook.

In [5]:
plot = k3d.plot(axes=['\\alpha', '\\beta', '\\gamma'])
plot.display()

Output()

The `axes_helper` attribute in K3D controls the visibility and size of the built-in coordinate axes displayed in the plot. By setting `plot.axes_helper = 0`, the axes helper is hidden, while `plot.axes_helper = 1` enables it with a default size. The example below uses `ipywidgets` and the `@interact` decorator to create an interactive slider that allows the user to dynamically adjust the size of the axes helper in real time.

In [7]:
plot.axes_helper = 0

In [9]:
plot.axes_helper = 1

In [11]:
from ipywidgets import interact
import ipywidgets as widgets

@interact(size=widgets.FloatSlider(min=0.0, max=200, value=1.0))
def g(size):
    plot.axes_helper=size

interactive(children=(FloatSlider(value=1.0, description='size', max=200.0), Output()), _dom_classes=('widget-…

In [13]:
plot.axes_helper = 3

In [15]:
plt = k3d.text('Text')
plt

Output()

In [17]:
plt.text = '\int_0^2 n y+ \sin x'

<>:1: SyntaxWarning: invalid escape sequence '\i'
<>:1: SyntaxWarning: invalid escape sequence '\i'
C:\Users\arija\AppData\Local\Temp\ipykernel_9856\1557127344.py:1: SyntaxWarning: invalid escape sequence '\i'
  plt.text = '\int_0^2 n y+ \sin x'


## Camera Manipulation in K3D

K3D allows users to interactively manipulate the camera view within a 3D scene, enabling rotation, zoom, and panning using the mouse. Beyond these interactive controls, the camera can also be programmatically controlled through the `plot.camera` property. This property accepts a list of nine values that define the camera's configuration: the first three values specify the camera position, the next three define the point it is looking at (target), and the final three represent the up direction vector. By adjusting these parameters, users can script precise camera views, simulate camera motion, or align the view to follow moving objects. Additionally, camera auto-fitting can be controlled with `plot.camera_auto_fit`, which when disabled allows full manual control over the camera's state. This capability is particularly important in applications such as simulation playback, animations, and camera path visualization in robotics or cinematography.

### Example 1. Visualizing Camera and Object Trajectories in 3D

The following example creates a 3D visualization that simulates and displays the trajectories of a camera and an object moving through space. First, a `k3d.plot()` is initialized, and a single arrow is added using `k3d.vectors` to represent a direction in space from the origin to the point (1, 1, 1).

Two trajectories are then generated:
- `cam_traj`: the camera trajectory, forming a spiral-like path in 3D space.
- `object_traj`: the object's trajectory, which follows a wavy and non-linear path.

Both trajectories are sampled with 100 points (`N = 100`) and visualized using `k3d.line`. The camera trajectory is shown with the default color and a simple shader, while the object trajectory is colored red (`0xff0000`) and rendered with a mesh shader for a more pronounced appearance.

The current positions of the camera and the object are also marked using `k3d.points`, initialized at the same starting point `[0, -1, 0]`. These can later be updated dynamically to animate movement along their respective paths.

This setup is particularly useful for visualizing motion planning, path-following, or camera tracking scenarios in robotics, animation, or simulation tasks. 

(Blue line is a camera path and the object is moving on the red line.)

In [19]:
plot = k3d.plot()

plt_arrow = k3d.vectors([0,0,0],[1,1,1])
plot += plt_arrow

N = 100
cam_traj = [np.array([-sin(phi), -cos(phi), phi*0.15],dtype=np.float32) for phi in np.linspace(0,2*pi,N)]
object_traj = [np.array([sin(phi),-1+cos(phi)+sin(2*phi),0.03*phi],dtype=np.float32) for phi in np.linspace(0,2*pi,N)]

plt_cam_traj = k3d.line(cam_traj ,shader='simple')
plt_object_traj = k3d.line(object_traj, color=0xff0000,shader='mesh')

plt_cam_pos = k3d.points([[0, -1, 0]],point_size=0.1,shader='mesh')
plt_object_pos = k3d.points([[0, -1, 0]],color=0xff0000,point_size=0.1,shader='mesh')

plot += plt_cam_traj
plot += plt_object_traj
plot += plt_object_pos
plot += plt_cam_pos

In [21]:
plot

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

### Example 2. Visualization of a 3D Point and Wireframe Cube with Coordinate Axes

This example demonstrates how to construct a 3D reference scene in K3D that includes coordinate axes, a spatial marker (point), and a manually defined wireframe cube. First, the X, Y, and Z axes are visualized using three color-coded lines—red for X, green for Y, and blue for Z—originating from the coordinate origin (0, 0, 0). A yellow point is then added at position (1, 2, 3) to mark a specific location in space.

To further enrich the scene, a cube centered at (1, 2, 3) with side length 1.0 is created manually by defining its eight vertices and connecting them using line segments to form the cube’s twelve edges. The cube is rendered as a cyan wireframe using `k3d.line`, giving a clear visual reference of a volumetric object in space. This type of manual geometry construction is valuable for understanding spatial relationships, testing transformation logic, or creating visual aids for educational and simulation purposes.


In [23]:
plot = k3d.plot()

# Draw Axes (X - Red, Y - Green, Z - Blue)
plot += k3d.line(np.array([[0, 0, 0], [1, 0, 0]]), color=0xff0000, width=0.01)  # X
plot += k3d.line(np.array([[0, 0, 0], [0, 1, 0]]), color=0x00ff00, width=0.01)  # Y
plot += k3d.line(np.array([[0, 0, 0], [0, 0, 1]]), color=0x0000ff, width=0.01)  # Z

# Add a point at (1, 2, 3)
plot += k3d.points(positions=np.array([[1, 2, 3]]), point_size=0.2, color=0xffff00)

# Define cube vertices (side = 1.0)
s = 0.5
cx, cy, cz = 1.0, 2.0, 3.0  # Center of cube
vertices = np.array([
    [cx - s, cy - s, cz - s],
    [cx + s, cy - s, cz - s],
    [cx + s, cy + s, cz - s],
    [cx - s, cy + s, cz - s],
    [cx - s, cy - s, cz + s],
    [cx + s, cy - s, cz + s],
    [cx + s, cy + s, cz + s],
    [cx - s, cy + s, cz + s]
])

# Define cube edges by connecting vertex indices
edges = [
    [0,1], [1,2], [2,3], [3,0],
    [4,5], [5,6], [6,7], [7,4],
    [0,4], [1,5], [2,6], [3,7]
]

# Create lines for the cube edges
for edge in edges:
    line = k3d.line(vertices[edge], width=0.01, color=0x00ffff)
    plot += line

plot.display()

C:\Users\arija\anaconda3\Lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "int32" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
C:\Users\arija\anaconda3\Lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

### Example 3. Animation Using Transformation Matrices in K3D

The following example illustrates how to animate a 3D object using a series of transformation matrices in K3D. A sequence of 4×4 model matrices is generated to represent rotation around the Z-axis over time. The rotation angle is calculated as a function of time using a full 360° (2π radians) cycle divided into 60 discrete steps. Each matrix not only performs the rotation but also includes a translation along the X-axis by 1 unit, allowing the object to orbit around the origin instead of rotating in place.

A single point located at (1, 0, 0) is created and assigned the first model matrix. The K3D plot is then displayed, and the object’s transformation is updated in a loop by assigning each matrix in the sequence to its `model_matrix` attribute. This results in a smooth, animated motion of the point tracing a circular path in the XY-plane. The example showcases how transformation matrices can be used in K3D to simulate object motion and is particularly useful for understanding rotational kinematics, orbital motion, and animation principles in 3D graphics.


In [25]:
# Create 60 evenly spaced time steps between 0 and 1
times = np.linspace(0, 1, 60)

# Initialize an empty list to store transformation (model) matrices
model_matrices = []

# Loop over each time step to create a rotation matrix
for t in times:
    angle = 2 * math.pi * t  # Compute rotation angle (in radians), completing one full circle (0 to 2π)

    # Define a 4x4 transformation matrix:
    # - Applies rotation in the XY-plane
    # - Translates the object 1 unit along the X-axis
    matrix = np.array([
        [math.cos(angle), -math.sin(angle), 0.0, 1.0],  # First row: X-axis direction + translation on X
        [math.sin(angle),  math.cos(angle), 0.0, 0.0],  # Second row: Y-axis direction
        [0.0,              0.0,             1.0, 0.0],  # Third row: Z-axis (no change)
        [0.0,              0.0,             0.0, 1.0]   # Fourth row: homogeneous coordinate
    ])

    # Store the matrix in the list for later use in animation
    model_matrices.append(matrix)

# Create a 3D point at position (1, 0, 0), with a visible size
obj = k3d.points(np.array([[1, 0, 0]]), point_size=0.5)

# Apply the first transformation matrix to the point (initial state)
obj.model_matrix = model_matrices[0]

# Create a new K3D plot and add the point object to it
plot = k3d.plot()
plot += obj

# Display the plot in the Jupyter notebook
plot.display()

import time  # Import time module to control animation speed

# Animate the point by applying each transformation matrix in sequence
for matrix in model_matrices:
    obj.model_matrix = matrix  # Update the transformation of the object
    time.sleep(0.05)           # Wait 50 milliseconds between frames to simulate motion

Output()

### Example 4. Combine Multiple Objects with Different Transformations

This example demonstrates how to display multiple 3D objects within the same scene, each with a different transformation applied. A red point is placed at a fixed position, while a green point is defined at the same initial location but transformed using a rotation matrix. Specifically, a 90-degree rotation around the Z-axis is applied to the green point by setting its `model_matrix`, which results in its position being visually rotated relative to the red point. This allows for a clear comparison between the original and transformed object. Combining multiple transformed objects in a shared coordinate system is essential in 3D graphics for tasks such as object manipulation, hierarchical modeling, and illustrating the effects of spatial transformations.


In [27]:
plot = k3d.plot()

# Add a red point at position (1, 0, 0) – this is the original untransformed point
plot += k3d.points(np.array([[1, 0, 0]]), point_size=0.2, color=0xff0000)

# Define a rotation angle of 90 degrees (π/2 radians) around the Z-axis
angle = math.pi / 2
# Create a 4x4 transformation matrix for rotation around the Z-axis
# This matrix will rotate points counter-clockwise in the XY plane
model_matrix = np.array([
    [math.cos(angle), -math.sin(angle), 0, 0],
    [math.sin(angle),  math.cos(angle), 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
], dtype=np.float32)

# Create a green point at the same initial position (1, 0, 0)
rotated_point = k3d.points(np.array([[1, 0, 0]]), point_size=0.2, color=0x00ff00)
# Apply the rotation matrix to the green point
# This rotates it to the new position (0, 1, 0), a 90° rotation around Z
rotated_point.model_matrix = model_matrix
# Add the rotated point to the plot
plot += rotated_point
# Display the interactive 3D scene
plot.display()

Output()

### Task 1. 3D Scene Composition with Camera and Axes
Build a simple 3D scene that includes:
- Axes helper
- A few geometric shapes placed in different positions
- A camera that you manipulate to explore the scene
- Apply a simple animation to one object using a transformation matrix (e.g. rotating or translating).

In [69]:
## Write your code here: 

## Part 2: Visualizing a Simple Point Cloud

Point clouds are one of the most fundamental data structures in 3D graphics, used to represent collections of points in space without explicit connectivity between them. In this example, a point cloud is created by generating 1,000 random 3D coordinates within a cube centered at the origin. These coordinates are passed to the `k3d.points` function, which visualizes them as small, uniformly sized red dots in the 3D space. By displaying this point cloud in an interactive K3D plot, users can rotate, zoom, and pan the view to gain an intuitive understanding of the spatial distribution of points. This kind of visualization is essential in areas such as 3D scanning, LiDAR data analysis, and early stages of 3D reconstruction pipelines.


In [29]:
# Generate random 3D points in a cube around origin
positions = np.random.uniform(-1, 1, (1000, 3))

# Create a point cloud plot
plot = k3d.plot()
plot += k3d.points(positions.astype(np.float32), point_size=0.1, color=0xff0000)
plot.display()

Output()

## Example 5. Colored Point Cloud Using Scalar Field

In this example, a structured point cloud is generated to form the surface of a sphere using spherical coordinates. Each point is then assigned a color based on its Z-coordinate (height), effectively visualizing a scalar field on the surface of the sphere. The scalar values are normalized and mapped to RGB color values using a simple gradient. This technique is useful in scientific visualization where scalar fields—such as temperature, elevation, or pressure—are represented through color. By encoding scalar information directly into the point cloud, users can intuitively understand patterns and variations across the dataset through interactive 3D exploration.


In [31]:
# Generate structured points on a sphere
phi = np.linspace(0, np.pi, 40)
theta = np.linspace(0, 2 * np.pi, 80)
phi, theta = np.meshgrid(phi, theta)

x = np.sin(phi) * np.cos(theta)
y = np.sin(phi) * np.sin(theta)
z = np.cos(phi)

positions = np.stack([x.ravel(), y.ravel(), z.ravel()], axis=1)

# Use Z values to color the points (height-based)
colors = ((z.ravel() + 1) / 2 * 0xFFFFFF).astype(np.uint32)

plot = k3d.plot()
plot += k3d.points(positions.astype(np.float32), colors=colors, point_size=0.05)
plot.display()


Output()

## Example 6. Interactive Voxel Grid – Binary Cube

This example demonstrates how to create a simple 3D voxel grid using K3D. A voxel grid is a 3D equivalent of a 2D pixel grid, where each cell (voxel) represents a value in a volumetric space. Here, a 5×5×5 grid of zeros is defined using a NumPy array, with a single voxel at the center set to 1. This active voxel is visualized using the `k3d.voxels` function, which renders a cube at the corresponding position within the grid. This type of binary voxel visualization is useful for representing discrete 3D structures, such as occupancy maps in robotics, binary segmentation masks in medical imaging, or early volumetric modeling in computer graphics.


In [33]:
# Create a 3D grid (5x5x5) with one filled cube in the middle
voxels = np.zeros((5, 5, 5), dtype=np.uint8)
voxels[2, 2, 2] = 1

plot = k3d.plot()
plot += k3d.voxels(voxels, bounds=[0, 5, 0, 5, 0, 5])
plot.display()


Output()

## Example 7. Thresholding a Volume to Voxels

This example illustrates how to generate a 3D volumetric dataset and convert it into a voxel representation using thresholding. A synthetic 3D Gaussian blob is created by evaluating the Gaussian function over a 3D grid of points centered in the volume. The resulting scalar field represents intensity values throughout the space. To visualize the structure, a threshold is applied to convert the continuous values into a binary volume—voxels with values above the threshold are set to 1, and others to 0. This binary volume is then rendered using `k3d.voxels`, displaying the 3D shape formed by the thresholded region. This approach is commonly used in medical imaging, CT/MRI segmentation, and scientific simulations to extract meaningful structures from scalar volumetric data.


In [35]:
# Create a 3D Gaussian blob
x, y, z = np.indices((20, 20, 20))
center = np.array([10, 10, 10])
sigma = 5
blob = np.exp(-((x - center[0])**2 + (y - center[1])**2 + (z - center[2])**2) / (2 * sigma**2))

# Threshold to binary volume
voxels = (blob > 0.1).astype(np.uint8)

plot = k3d.plot()
plot += k3d.voxels(voxels, bounds=[0, 20, 0, 20, 0, 20])
plot.display()


Output()

## Example 8. Animate Point Cloud Change Over Time

This example showcases how to animate a 3D point cloud by dynamically updating its positions frame by frame. An initial set of random points is generated in 3D space and visualized using `k3d.points`. In a loop, small random displacements are added to each point's position to simulate motion, and the `positions` attribute of the point cloud object is updated accordingly. This creates a simple but effective animation, where the points appear to drift or jitter over time. Animating point clouds is useful for visualizing temporal phenomena, such as particle simulations, fluid motion, dynamic systems, or evolving datasets in scientific and engineering applications.


In [37]:
import time

# Initial random positions
positions = np.random.uniform(-1, 1, (200, 3)).astype(np.float32)
plot = k3d.plot()
points = k3d.points(positions, point_size=0.1, color=0x00ff00)
plot += points
plot.display()

# Animate a change in positions
for _ in range(50):
    positions += np.random.normal(scale=0.01, size=positions.shape).astype(np.float32)
    points.positions = positions
    time.sleep(0.05)


Output()

## Example 9. Point Cloud Compression via Grid Downsampling

In this example, a large and dense point cloud is simplified using a clustering-based downsampling technique. A set of 10,000 randomly distributed points is generated, representing a complex or high-resolution dataset. To reduce its size while preserving its overall structure, KMeans clustering is applied to group the points into a smaller number of representative clusters. The cluster centers are then visualized as a new, compressed point cloud using `k3d.points`. This method significantly reduces the number of points, enabling faster rendering and easier analysis, while maintaining the geometric characteristics of the original data. Point cloud compression is essential in applications like real-time rendering, mobile 3D visualization, and preprocessing for machine learning models.


In [39]:
from sklearn.cluster import KMeans  # Import the KMeans clustering algorithm from scikit-learn

# Generate a dense 3D point cloud of 10,000 points
# Each point has random coordinates in the range [-1, 1] for x, y, and z
positions = np.random.uniform(-1, 1, (10000, 3)).astype(np.float32)

# Apply KMeans clustering to compress the dataset
# Group the 10,000 points into 300 clusters
kmeans = KMeans(n_clusters=300)

# Fit the KMeans model to the original point cloud
kmeans.fit(positions)

# Get the cluster centers — these represent the "compressed" version of the data
compressed = kmeans.cluster_centers_

# Create a K3D plot to visualize the compressed point cloud
plot = k3d.plot()

# Add the cluster centers to the plot as yellow-orange points
plot += k3d.points(compressed.astype(np.float32), point_size=0.15, color=0xffaa00)

# Display the interactive 3D scene
plot.display()

Output()

## Task 2: Build a Rotating Object Group (Hierarchical Transformation)

Create a scene with at least three points or geometric shapes that rotate together around a shared axis, simulating a hierarchical transformation (like satellites orbiting a planet).
Steps:
- Place one central object (e.g. a large white sphere or point).
- Place 2–3 smaller objects at fixed offsets from the central object.
- Apply a rotation matrix over time to all satellite objects around the central one.
- Use different rotation speeds or radii.
Expected Outcome: A dynamic orbital system where all satellites rotate around the central object while maintaining their relative spacing.

## Task 3: Dynamic Point Cloud with Real-Time Clustering Visualization

Visualize a dynamic point cloud that updates over time and display the real-time cluster centers using KMeans.
Steps: 
- Start with ~500 randomly distributed 3D points.
- At each animation step:
- Slightly move all points (simulate motion).
- Re-run KMeans clustering (e.g. n_clusters=10).
- Update both the moving point cloud and the cluster centers (as larger points or differently colored).
- Color each point by its cluster label.

Expected Outcome: A "living" point cloud that moves and updates, with cluster centers tracking the centroid of nearby points.

## Task 4: Create a Procedural Voxel Scene with Multiple Thresholded Volumes

Generate and combine multiple scalar fields (e.g., Gaussian blobs, sinusoids), apply different thresholds to each, and render them together using voxels in different colors.
Steps: 
- Generate two or more 3D scalar fields (e.g., blob1, blob2).
- Threshold each scalar field independently with different values.
- Combine their binary masks, ensuring no overlap or blending.
- Assign each voxel group a distinct color using the color_map argument in k3d.voxels.
- Animate the thresholds over time and observe shape morphing.

Expected Outcome: A multi-object volumetric scene where students explore scalar-to-voxel conversion, boolean logic on volumes, and multi-object 3D rendering.